# NaiveBayes 과제

#### 개요
1. Multinomianl Naïve Bayes 예제로 데이터에서 Test값은 몇 % 확률로 어떤 클래스로 분류될까요? 
2. 실습코드 2번째 코드는 미완성의 코드입니다. 나이브 베이즈 클래시파이어를 구현 했지만 몇 가지 문제점을 가지고 있습니다.
 * 첫 째, 너무 많은 확률값을 가지고 있습니다. 
 * 둘 째, 값이 0에 수렴할 수 있습니다. (어떤 기법을 적용해서 해결하면 좋을까요?)
      
   해당 문제를 코딩으로 구현하고 설명해주세요. e.g 너무 많은 확률 값을 가지고 있기에 00을 적용했습니다. 
   
#### 데이터 출처

1번 - 강의자님 제공\
2번 - German Credit Dataset

In [1]:
# cleaning texts 
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score 
import math

# 과제 1

In [2]:
# ["Chinese Chinese Chinese Tokyo Janpan", "?"]
  
dataset = [["Chinese Beijing Chiense", "c"], 
           ["Chinese Chinese Shanghai", "c"], 
           ["Chinese Macao", "c"],
           ["Tokyo Japan Chinese", "j"]] 
              
dataset = pd.DataFrame(dataset) 
dataset.columns = ["Text", "Class"]

cv = CountVectorizer() 
y = dataset['Class'].values 
X = cv.fit_transform(dataset.Text)
feature_df = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())
feature_df

,beijing,chiense,chinese,japan,macao,shanghai,tokyo
0,1,1,1,0,0,0,0
1,0,0,2,0,0,1,0
2,0,0,1,0,1,0,0
3,0,0,1,1,0,0,1


* dataset에서 Text, Class로 구분을 해줍니다
* CountVectorizer로 전처리를 해줍니다. 단어들을 토큰화시켜서 나이브 베이즈 분류를 해주기위해서는 데이터를 BoW로 만들어줄 필요가 있습니다.
* 텍스트를 자동으로 BoW로 만드는 CountVectorizer를 사용하겠습니다.

In [3]:
mod = MultinomialNB()
mod.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

* 다항분포 나이브 베이즈 모델 클래스를 사용합니다. 파라밍터의 alpha=1.0은 라플라스 스무딩이 사용되었음을 의미합니다.

In [4]:
test = ["Chinese Chinese Chinese Tokyo Janpan"]
test_cv = cv.transform(test)
feature_df = pd.DataFrame(test_cv.toarray(), columns=cv.get_feature_names())
feature_df

,beijing,chiense,chinese,japan,macao,shanghai,tokyo
0,0,0,3,0,0,0,1


In [5]:
predicted = mod.predict(test_cv) 
prob = np.max(mod.predict_proba(test_cv))
print(f'예측 = {predicted[0]}, 확률 = {prob}')

예측 = c, 확률 = 0.8223684210526319


* test 문장을 기존의 cv에 transform해주고 확인을 해보니 토큰화가 잘 되었습니다.
* mod에 test문장을 인자로 넣어주어 모델의 출력값인 class를 predicted에 넣어줍니다.
* predict_proba로 확률값 또한 받아주는데, 각 클래스 별 확률이 numpy array로 반환됩니다. 따라서 제일 높은 값을 max로 받아와서 prob에 넣어줍니다.


# 과제2

* German Credit Dataset


In [6]:
data_url = '../data/fraud.csv'
df = pd.read_csv(data_url, sep=',')
df.head()

,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [7]:
del df["ID"] 
Y_data = df.pop("Fraud")
Y_data = Y_data.values
Y_data

array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

* ID는 분석에 의미가 없으니 제거, 타겟 데이터인 Fraud값을 따로 떼어서 저장합니다.
* 범주형 변수들을 더미로 만들어주겠습니다.

In [8]:
x_df = pd.get_dummies(df)
x_df.head() 

,History_arrears,History_current,History_none,History_paid,CoApplicant_coapplicant,CoApplicant_guarantor,CoApplicant_none,Accommodation_free,Accommodation_own,Accommodation_rent
0,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,1,0,1,0
2,0,0,0,1,0,0,1,0,1,0
3,0,0,0,1,0,1,0,0,0,1
4,1,0,0,0,0,0,1,0,1,0


In [9]:
x_array = x_df.values
x_array

array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=uint8)

## Q1. as_matrix()함수를 통해 우리가 하고자 하는 것은 무엇일까요?
* 나이브 베이즈를 통해 as_matrix로 만든 더미와 타겟 데이터들을 가지고 분류를 하려고 합니다. 

* pandas DataFrame 자체로는 모델에 넣을 수 없기 때문에 numpy 타입으로 바꿔줍니다.

In [71]:
# 1) sklearn
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(x_array, Y_data)
print(np.exp(model.class_log_prior_))

# 2) 손코딩
P_Y_True = sum(Y_data) / len(Y_data) # True 개수 / label set length 
P_Y_False = 1 - P_Y_True # False 개수 / label set length
P_Y_False, P_Y_True

[0.7 0.3]


(0.7, 0.3)

In [72]:
# 각 Boolean 값에 해당하는 index 가져오기 
ix_Y_True = np.where(Y_data)
ix_Y_False = np.where(Y_data == False)

ix_Y_True, ix_Y_False

((array([ 0,  3,  5,  9, 11, 12], dtype=int64),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19],
        dtype=int64),))

In [87]:
# P(Xi|Yc) 
# p_x_y_true = (x_array[ix_Y_True].sum(axis=0) / sum(Y_data==True))  # Q.뒤에 sum(Y_data == True) 필요한가요? 각 xi에 따른 확률 값도 확인 할 수 있지 않나요?
# p_x_y_true = ( model.feature_count_[1] / sum(Y_data))
# 뒤에 trainset에서 각 클래스 빈도를 넣음 sum(y_c) 

p_x_y_true = (x_array[ix_Y_True].sum(axis=0) / sum(Y_data==True))
p_x_y_false = (x_array[ix_Y_False].sum(axis=0) / sum(Y_data==False))


# productP(X|Yc) with sklearn
# 뒤에 각 클래스 별 feature들의 빈도의 sum을 넣음
# 수업자료에 sum of all term freq in y_c라고 되어있는 것으로 봐선 이쪽이 맞지 않나?

fc = model.feature_count_
product_ = fc / fc.sum(axis=1)[:,None] # == fc / np.repeat(fc.sum(axis=1)[:, np.newaxis], 10, axis=1)
p_x_y_true = product_[1]
p_x_y_false = product_[0]

print(p_x_y_true)
print(p_x_y_false)

[0.05555556 0.16666667 0.05555556 0.05555556 0.         0.05555556
 0.27777778 0.         0.22222222 0.11111111]
[0.14285714 0.0952381  0.         0.0952381  0.04761905 0.
 0.28571429 0.02380952 0.26190476 0.04761905]


* 클래스와 특성의 값이 훈련 데이터에서 발생 하지 않는 경우에는, 빈도수 기반의 확률 추정치는 0이 됩니다. 이것은 이 추정치가 곱해질 때 다른 확률의 모든 정보를 없앨 수 있기 때문에 문제가 될 수 있습니다. 따라서, 대부분의 확률 추정에서는 그 값이 0이 되지 않도록 정규화 시켜주는데, 가짜 수(pseudocount) 1을 보정값을 주는 방법을 라플라스 정규화(Laplace smoothing)라고 불립니다.

* 간단하게 확률 값을 모두 곱하는 방식은 0에 가까워지면서 언더플로 문제가 발생합니다. 따라서 log를 취해주어 바꾸어 줄 수 있습니다.

In [99]:
# 1) sklearn
theta = np.exp(model.feature_log_prob_)
print(theta)


# model.alpha = 1.0
theta = (fc + model.alpha) / (fc.sum(axis=1)[:,None] + model.alpha * x_array.shape[1])
p_x_y_true = theta[1]
p_x_y_false = theta[0]

print(p_x_y_false)
print(p_x_y_true)

[[0.13461538 0.09615385 0.01923077 0.09615385 0.05769231 0.01923077
  0.25       0.03846154 0.23076923 0.05769231]
 [0.07142857 0.14285714 0.07142857 0.07142857 0.03571429 0.07142857
  0.21428571 0.03571429 0.17857143 0.10714286]]
[0.13461538 0.09615385 0.01923077 0.09615385 0.05769231 0.01923077
 0.25       0.03846154 0.23076923 0.05769231]
[0.07142857 0.14285714 0.07142857 0.07142857 0.03571429 0.07142857
 0.21428571 0.03571429 0.17857143 0.10714286]


In [104]:
#Log(P(Fraud|features)) = log(P(History_arrears|Fraud))+log(P(CoApplicant_coapplicant|Fraud)+…+log(P(Fraud))
x_test = [0,1,0,0,0,1,0, 0,1,0]

p_y_true_test = np.log(P_Y_True) + np.log(p_x_y_true.dot(x_test))
p_y_false_test = np.log(P_Y_False) + np.log(p_x_y_false.dot(x_test))

p_y_true_test, p_y_false_test

(-2.1382820417027695, -1.4175469046239952)

* 테스트 데이터를 넣어보니 거짓으로 구분했습니다.

## ref
* https://datascienceschool.net/view-notebook/c19b48e3c7b048668f2bb0a113bd25f7/
* https://ko.wikipedia.org/wiki/%EB%82%98%EC%9D%B4%EB%B8%8C_%EB%B2%A0%EC%9D%B4%EC%A6%88_%EB%B6%84%EB%A5%98
* https://bcho.tistory.com/1010
* https://kenzotakahashi.github.io/naive-bayes-from-scratch-in-python.html